In [1]:
import os
import numpy as np
import pandas as pd
os.chdir("/content/drive/MyDrive/cs289/proj/code")

In [5]:
pip install pysolnp

     |████████████████████████████████| 914 kB 4.1 MB/s 


In [2]:
"""
Test 1: data process
"""
from data_process import data_prep_task
X_t, X_v, Y_t, Y_v = data_prep_task('wash', test_size=0.3)

Categorical features imputed successfully.
Discrete numerical features imputed successfully.
Int values turned into float type successfully.
Continuous numerical features imputed successfully.
Dataset is standardized.
Data pre-processing is finished.


In [3]:
X_t = X_t[:, 0:16]
X_t = X_t.astype(float)

In [6]:
"""
Test 2: learners
"""
from learners import *
from sl_task import sl_task

In [7]:
# define task
task = sl_task(X = X_t, Y = Y_t, family = "Gaussian", K = 10)

# define candidate learners
L_glm = Lrnr_glm(sl_task = task)
L_lasso = Lrnr_glmnet(sl_task = task)
L_rf = Lrnr_rf(sl_task = task)
L_xgb = Lrnr_xgboost(sl_task = task, num_round = 200)

In [7]:
# test learners individually
# - glm
L_glm.train(Y = Y_t, X = X_t)
y_hat = L_glm.predict(X = X_t)
mse1 = square_error_loss(y_hat, Y_t)

# - glmnet
L_lasso.train(Y = Y_t, X = X_t)
y_hat = L_lasso.predict(X = X_t)
mse2 = square_error_loss(y_hat, Y_t)

# - randomforest
L_rf.train(Y = Y_t, X = X_t)
y_hat = L_rf.predict(X = X_t)
mse3 = square_error_loss(y_hat, Y_t)

# - xgboost
L_xgb.train(Y = Y_t, X = X_t)
y_hat = L_xgb.predict(X = X_t)
mse4 = square_error_loss(y_hat, Y_t)

In [8]:
"""
Test 3: screener, pipeline
"""
from screeners import Screen_univar
from pipeline import Pipeline

In [9]:
# - screener univariate
S_uni = Screen_univar(sl_task = task, k = 5)
# S_uni.train(Y = Y_t, X = X_t)

# - pipepline
L_xgb100 = Lrnr_xgboost(sl_task = task, num_round = 100)
learners = [S_uni, L_xgb100]
L_pipe = Pipeline(sl_task = task, learners = learners)

# L_pipe.train(Y = Y_t, X = X_t)
# y_hat = L_pipe.predict(X_t)
# mse5 = square_error_loss(y_hat, Y_t)

In [14]:
"""
Test 4: sl
"""

# Step 1. define task
task = sl_task(X = X_t, Y = Y_t, family = "Gaussian", K = 10)

# Step 2. define candidate learners
L_glm = Lrnr_glm(sl_task = task)
L_lasso = Lrnr_glmnet(sl_task = task)
L_rf = Lrnr_rf(sl_task = task)
L_xgb = Lrnr_xgboost(sl_task = task, num_round = 200)

S_uni = Screen_univar(sl_task = task, k = 5) # screener univariate
L_xgb100 = Lrnr_xgboost(sl_task = task, num_round = 100) # pipepline
learners = [S_uni, L_xgb100]
L_pipe = Pipeline(sl_task = task, learners = learners)

stack = [L_glm, L_lasso, L_rf, L_xgb, L_pipe]


In [17]:
# Step 3. initialize sl
sl = initialize_sl(sl_task = task, stack = stack, meta = "nnls")

sl.train()

Training on Fold 1 of 10 

Training on Fold 2 of 10 

Training on Fold 3 of 10 

Training on Fold 4 of 10 

Training on Fold 5 of 10 

Training on Fold 6 of 10 

Training on Fold 7 of 10 

Training on Fold 8 of 10 

Training on Fold 9 of 10 

Training on Fold 10 of 10 



In [18]:
# summary sl
sl.summary()

 Num. of cv folds:  10 
 Meta learner:  nnls 
 n:  3286 
 p:  16
  learner    weight   cv_risk
0     glm  0.204137  1.371883
1  glmnet  0.000000  1.370490
2      rf  0.872414  1.025892
3     xgb  0.109682  1.051700
4    pipe  0.000000  1.044191
